# Cafe Sales Data Cleaning


In [47]:
import pandas as pd
import numpy as np

## Loading the data


In [48]:
df = pd.read_csv('dirty_cafe_sales.csv')
df.shape

(10000, 8)

In [49]:
df.head(10)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NaN,2023-03-31
6,TXN_4433211,UNKNOWN,3,3.0,9.0,ERROR,Takeaway,2023-10-06
7,TXN_6699534,Sandwich,4,4.0,16.0,Cash,UNKNOWN,2023-10-28
8,TXN_4717867,NaN,5,3.0,15.0,NaN,Takeaway,2023-07-28
9,TXN_2064365,Sandwich,5,4.0,20.0,NaN,In-store,2023-12-31


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


I can see that Quantity, Price Per Unit and Total Spent are showing as object type instead of numbers. This is because there are ERROR and UNKNOWN strings in some rows. Also some columns have missing values.

## Handling invalid values
I will replace ERROR and UNKNOWN with NaN so I can convert the columns to the right types. After that I will try to recover the missing values using the other columns.

In [51]:
df.replace(['ERROR', 'UNKNOWN'], np.nan, inplace=True)

## Data type correction
Now I can convert the numeric columns and the date column to their proper types.

In [52]:
df['Quantity'] = pd.to_numeric(df['Quantity'])
df['Price Per Unit'] = pd.to_numeric(df['Price Per Unit'])
df['Total Spent'] = pd.to_numeric(df['Total Spent'])
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])

In [53]:
df.isnull().sum()

Transaction ID         0
Item                 969
Quantity             479
Price Per Unit       533
Total Spent          502
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64

## Recovering missing values
Before dropping any rows I want to try to fill in the missing values. Each item has a fixed price so I created a mapping for that. Some prices are unique to one item (like Coffee is always 2.0) so I can use the price to figure out the item name.

In [54]:
price_map = {'Coffee': 2.0, 'Tea': 1.5, 'Cookie': 1.0, 'Juice': 3.0, 'Cake': 3.0, 'Sandwich': 4.0, 'Smoothie': 4.0, 'Salad': 5.0}
item_map = {1.0: 'Cookie', 1.5: 'Tea', 2.0: 'Coffee', 5.0: 'Salad'}

I didnt include 3.0 and 4.0 in item_map because Juice and Cake both cost 3.0, and Sandwich and Smoothie both cost 4.0, so I cant tell which one it is from the price alone.

In [55]:
df.loc[df['Price Per Unit'].isna(), 'Price Per Unit'] = df['Item'].map(price_map)
df['Price Per Unit'].isna().sum()

54

In [56]:
df.loc[df['Item'].isna(), 'Item'] = df['Price Per Unit'].map(item_map)
df['Item'].isna().sum()

501

Now I will use the relationship Total Spent = Quantity * Price Per Unit to fill in the missing values for any of these three columns.

In [57]:
df.loc[df['Total Spent'].isna(), 'Total Spent'] = df['Quantity'] * df['Price Per Unit']
df['Total Spent'].isna().sum()

23

In [58]:
df.loc[df['Quantity'].isna(), 'Quantity'] = df['Total Spent'] / df['Price Per Unit']
df['Quantity'].isna().sum()

23

In [59]:
df.loc[df['Price Per Unit'].isna(), 'Price Per Unit'] = df['Total Spent'] / df['Quantity']
df['Price Per Unit'].isna().sum()

6

Running a second pass because now that we filled some Price values we might be able to recover more Item names and Total Spent values.

In [60]:
df.loc[df['Item'].isna(), 'Item'] = df['Price Per Unit'].map(item_map)
df.loc[df['Total Spent'].isna(), 'Total Spent'] = df['Quantity'] * df['Price Per Unit']

In [61]:
df.isnull().sum()

Transaction ID         0
Item                 480
Quantity              23
Price Per Unit         6
Total Spent           23
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64

## Handling remaining missing values
For Payment Method and Location I cant figure out the correct value from other columns so I will just fill them with "Unknown". For the rest of the columns if they are still missing I will drop those rows since there is no way to know what value they should have.

In [62]:
df['Payment Method'].fillna('Unknown', inplace=True)
df['Location'].fillna('Unknown', inplace=True)

In [63]:
df.dropna(subset=['Item', 'Quantity', 'Price Per Unit', 'Total Spent', 'Transaction Date'], inplace=True)
df.shape

(9064, 8)

In [64]:
df.isnull().sum()

Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64

## Adding the Season column
The assignment asks to add a Season column based on the month of the transaction date. I will use December-February as Winter, March-May as Spring, June-August as Summer and September-November as Fall.

In [65]:
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df['Season'] = df['Transaction Date'].apply(get_season)

In [66]:
df.head(10)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date,Season
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08,Fall
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16,Spring
2,TXN_4271903,Cookie,4.0,1.0,4.0,Credit Card,In-store,2023-07-19,Summer
3,TXN_7034554,Salad,2.0,5.0,10.0,Unknown,Unknown,2023-04-27,Spring
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11,Summer
5,TXN_2602893,Smoothie,5.0,4.0,20.0,Credit Card,Unknown,2023-03-31,Spring
7,TXN_6699534,Sandwich,4.0,4.0,16.0,Cash,Unknown,2023-10-28,Fall
9,TXN_2064365,Sandwich,5.0,4.0,20.0,Unknown,In-store,2023-12-31,Winter
10,TXN_2548360,Salad,5.0,5.0,25.0,Cash,Takeaway,2023-11-07,Fall
12,TXN_7619095,Sandwich,2.0,4.0,8.0,Cash,In-store,2023-05-03,Spring


In [67]:
df.shape

(9064, 9)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9064 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    9064 non-null   object        
 1   Item              9064 non-null   object        
 2   Quantity          9064 non-null   float64       
 3   Price Per Unit    9064 non-null   float64       
 4   Total Spent       9064 non-null   float64       
 5   Payment Method    9064 non-null   object        
 6   Location          9064 non-null   object        
 7   Transaction Date  9064 non-null   datetime64[ns]
 8   Season            9064 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 708.1+ KB


## Saving the cleaned data
save the cleaned dataframe to a new csv file.

In [69]:
df.to_csv('cleaned_cafe_sales.csv', index=False)